In [1]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 3.9MB/s 


In [2]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from nltk.corpus import stopwords
import pandas as pd
import pickle
import re
import os

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
## ADDITIONAL FUNCTIONS
if not os.path.exists('models'):
    os.makedirs('models')
    
pStopWordsList = stopwords.words('english') + stopwords.words('russian')

def SearchGoodsinTxt(ptxt, pGoodsPN, pGoodsDescription, pGoodsQty, pGoodsUnit, spaces=3):
    if (pGoodsPN==None)&(pGoodsDescription==None):
        return None
    if pGoodsPN in (None, ''): 
        pGoodsPN = '' 
    else: 
        pGoodsPN= '%s.{0,%s}' % (re.escape(pGoodsPN.lower()), spaces)
    if pGoodsDescription in (None, ''): 
        pGoodsDescription = '' 
    else: 
        pGoodsDescription= '%s.{0,%s}' % (re.escape(pGoodsDescription.lower()), spaces)
    if pGoodsQty in (None, ''): 
        pGoodsQty = '' 
    else: 
        pGoodsQty= '%s.{0,%s}' % (re.escape(pGoodsQty.lower()), spaces)
        
    if pGoodsUnit in (None, ''): 
        pGoodsUnit = '' 
    else: 
        pGoodsUnit= '%s.{0,%s}' % (re.escape(pGoodsUnit.lower()), spaces)
        
    regext = '%s%s%s%s' % (pGoodsPN, pGoodsDescription, pGoodsQty, pGoodsUnit)
    #print(regext)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsPN, pGoodsQty, pGoodsUnit)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsPN, pGoodsUnit,pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s%s%s%s' % (pGoodsPN, pGoodsDescription, pGoodsUnit,pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s%s%s%s' % (pGoodsQty, pGoodsUnit, pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsUnit, pGoodsQty, pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsUnit, pGoodsQty, pGoodsDescription,pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsQty, pGoodsUnit, pGoodsDescription,pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsPN, pGoodsQty, pGoodsUnit, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsQty, pGoodsUnit, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsPN, pGoodsUnit, pGoodsQty, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsUnit, pGoodsQty, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
   
    regext = '%s%s%s' % (pGoodsPN, pGoodsDescription, pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s' % (pGoodsDescription, pGoodsPN, pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s' % (pGoodsQty, pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s%s%s' % (pGoodsQty, pGoodsDescription, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    #return m
    regext = '%s%s' % (pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s' % ( pGoodsDescription, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s' % (pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s' % (pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    return m


def process_text_layer(TextLayer, pStopWordsList = []):
    txt =  " ".join([w for w in TextLayer.lower().split() \
                     if (not w in pStopWordsList)])

    txt = txt. \
        replace('«', ''). \
        replace('»', ''). \
        replace('(', ''). \
        replace(')', ''). \
        replace('\[)', ''). \
        replace('\]', ''). \
        replace('^', ''). \
        replace('\\', '')

    return txt

def tagging_sentence_by_targets(ptag, sentence_tagged, df_targets, pmodel_name):
    ret = sentence_tagged
    for r in df_targets[df_targets['tag']==ptag]['string_value']:
        tagging_sentence(ret, r, pmodel_name)
    
    return ret

def word2features2(sent, i, exclude = 'O', bias=3):
    word = sent[i][0]
    if sent[i][2] == exclude:
        return None

    features = {
        'bias': bias,
        'word.lower()': word.lower(),
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'BOS': False,
        'EOS': False
    }
    for w in range(1, bias+1):
        if i > w:
            word1 = sent[i-w][0]
            features.update({
                '-%s:word.lower()' % (w): word1.lower()
            })
            
    if i < 1:
        features['BOS'] = True

    for w in range(1, bias+1):
        if i < len(sent)-w:
            word1 = sent[i+w][0]
            features.update({
                '+%s:word.lower()' % (w): word1.lower()
            })
            
    if i >= len(sent)-1:
        features['EOS'] = True

    return features

def sent2features(sent, exclude = 'O'):
    result = list()
    for i in range(len(sent)):
        ret = word2features2(sent, i, exclude = exclude, bias=10)
        if ret !=None:
            result.append(ret)
    return result

def sent2labels(sent, exclude = 'O'):
    result = list()
    for token, postag, label in sent:
        if label !=exclude:
            result.append(label)        
    return result

def tagging_sentence(sentence, target, tag):
    sentence_text=  ' '.join([w['Word'] for w in sentence])
    r_tag = ''

    try:
        res_find  = re.finditer(target.lower(), sentence_text.lower())
    except:
        return sentence

    for m in res_find:
        r_tag = 'B-%s' % tag
        for i in range(len(sentence_text[:m.start()].split()), len(sentence_text[:m.end()].split())):
            """
            if r_tag==('B-%s' % tag):
                if i>0:
                    sentence[i-1]['Tag']= 'B1-%s' % tag
                if i>1:
                    sentence[i-2]['Tag']= 'B2-%s' % tag                      
                if i>2:
                    sentence[i-3]['Tag']= 'B3-%s' % tag
                if i>3:
                    sentence[i-4]['Tag']= 'B4-%s' % tag  
            """
            sentence[i]['Tag']= r_tag
            r_tag = 'I-%s' % tag   
            """
            for j in range(1,5):
                if i+j < len(sentence):
                    sentence[i+j]['Tag']='E%s-%s' % (str(j), tag)
            """
    return sentence

def prepare_sentence(num, sentence):
    words = sentence.replace('"', '').split()
    tmp = []
    for w in words:
        tmp.append({'Sentence #': 'Sentence: %s' % num, 'Word' : w, 'POS' : 'NN', 'Tag': 'O'})
    return tmp

In [5]:
!gdown --id 1mvcIhS_E4YhOMP3Tcm898aOjVDZIqLp-

Downloading...
From: https://drive.google.com/uc?id=1mvcIhS_E4YhOMP3Tcm898aOjVDZIqLp-
To: /content/NER RFQ.csv
5.41MB [00:00, 85.8MB/s]


In [6]:
data = pd.read_csv('NER RFQ.csv')
for column in data.columns:
    data[column] = data[column].astype(str)

In [7]:
GoodsString = data.copy()
GoodsString['tag'] = GoodsString.apply(lambda r: r['tag'].split('-')[0], axis =1)
GoodsString['string_value'] = GoodsString.apply(lambda r: process_text_layer(r['string_value'], pStopWordsList), axis = 1)

GoodsString['Match'] = GoodsString.apply(lambda r: SearchGoodsinTxt(r['string_value'], r['GoodsPN'], r['GoodsDescription'], r['GoodsQTY'], r['GoodsUnit'], 5), axis=1)

In [8]:
model_name = 'GoodsString'
df_x = GoodsString[GoodsString['Match'].notnull()][['tag', 'string_value']].rename(columns={'string_value':'x'})
df_y = GoodsString[GoodsString['Match'].notnull()][['tag', 'Match']].rename(columns={'Match': 'string_value'})
df_x["x_tagged"] = df_x.apply(lambda r: prepare_sentence(r.name, r["x"]), axis=1)
df_x["x_tagged"] = df_x.apply(lambda r: tagging_sentence_by_targets(r["tag"], r['x_tagged'], df_y, model_name), axis=1)

In [9]:
validation_set_part = 0.2

df_validation = df_x.sample(round(len(df_x)*validation_set_part)).copy()
df_training = df_x[df_x['tag'].isin(df_validation['tag'].values)==False].copy()

In [10]:
df_training.head()

,tag,x,x_tagged
70,119287R.msg,name 119287r.msg <aogdesk@nordstar.ru> <mvf@it...,"[{'Sentence #': 'Sentence: 70', 'Word': 'name'..."
76,120421R.msg,"name 120421r.msg ""d.klebcha@s7.ru"" <d.klebcha@...","[{'Sentence #': 'Sentence: 76', 'Word': 'name'..."
88,120660R.msg,"name 120660r.msg ""yuliya a. kondratova"" <yuliy...","[{'Sentence #': 'Sentence: 88', 'Word': 'name'..."
89,120660R.msg,"name 120660r.msg ""yuliya a. kondratova"" <yuliy...","[{'Sentence #': 'Sentence: 89', 'Word': 'name'..."
90,120660R.msg,"name 120660r.msg ""yuliya a. kondratova"" <yuliy...","[{'Sentence #': 'Sentence: 90', 'Word': 'name'..."


In [11]:
df_training["tags"] = df_training.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
df_training["words"] = df_training.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])

In [12]:
df_training = df_training.loc[:, ['tag', 'words', 'tags']].rename(columns={'tag' : 'id', 'tags' : 'ner_tags', 'words' : 'tokens'})
df_training.head()

,id,tokens,ner_tags
70,119287R.msg,"[name, 119287r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
76,120421R.msg,"[name, 120421r.msg, d.klebcha@s7.ru, <d.klebch...","[O, O, O, O, O, O, O, O, B-GoodsString, I-Good..."
88,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
89,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
90,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [13]:
df_validation["tags"] = df_validation.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
df_validation["words"] = df_validation.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])

In [14]:
df_validation = df_validation.loc[:, ['tag', 'words', 'tags']].rename(columns={'tag' : 'id', 'tags' : 'ner_tags', 'words' : 'tokens'})
df_validation.head()

,id,tokens,ner_tags
463,123312R.msg,"[name, 123312r.msg, natallia.baboryk, <natalli...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1728,125028R.msg,"[name, 125028r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
852,123781R.msg,"[name, 123781r.msg, <nataliya.zhmurko@utair.ru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1648,124922R.msg,"[name, 124922r.msg, <gennadiy.moskatov@utair.r...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
990,123937R.msg,"[name, 123937r.msg, <gennadiy.moskatov@utair.r...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [15]:
print(df_training.shape, df_validation.shape)

(811, 3) (385, 3)


In [16]:
df = pd.concat([df_training, df_validation])
df.head()

,id,tokens,ner_tags
70,119287R.msg,"[name, 119287r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
76,120421R.msg,"[name, 120421r.msg, d.klebcha@s7.ru, <d.klebch...","[O, O, O, O, O, O, O, O, B-GoodsString, I-Good..."
88,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
89,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
90,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [17]:
df.to_csv("NER_RFQ_agg.csv", index=False)

In [18]:
all_tags = set()
word_to_ix = {}
for _, row in df.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
    for word in row.tokens:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
all_tags = sorted(list(all_tags))
tag_to_idx = {t: i for i, t in enumerate(all_tags)}
label_list = {i: t for i, t in enumerate(all_tags)}

In [19]:
def find_token(x):
    res = []
    for w in x['tokens']:
        if w in word_to_ix.keys():
            res.append(word_to_ix[w])
        else:
            res.append(0)
    return res

df['encoded_ner_tags'] = df.apply(lambda x: [tag_to_idx[t] for t in x['ner_tags']], axis=1)
df['int_tokens'] = df.apply(find_token, axis=1)

In [20]:
df

,id,tokens,ner_tags,encoded_ner_tags,int_tokens
70,119287R.msg,"[name, 119287r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9, 11, 12, ..."
76,120421R.msg,"[name, 120421r.msg, d.klebcha@s7.ru, <d.klebch...","[O, O, O, O, O, O, O, O, B-GoodsString, I-Good...","[2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, ...","[0, 72, 73, 74, 75, 76, 5, 77, 78, 79, 80, 8, ..."
88,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 116, 117, 118, 119, 120, 70, 5, 77, 121, 1..."
89,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 116, 117, 118, 119, 120, 70, 5, 77, 121, 1..."
90,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 116, 117, 118, 119, 120, 70, 5, 77, 121, 1..."
...,...,...,...,...,...
2095,125492R.msg,"[name, 125492r.msg, mirgul, sadykova, <mirgul....","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 10885, 1219, 1220, 1221, 70, 5, 10886, 8, ..."
602,123436R.msg,"[name, 123436r.msg, <sales06@yestechnik.com>, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 9633, 3905, 4, 5, 9634, 8, 9, 3907, 312, 1..."
795,123711R.msg,"[name, 123711r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 12355, 3351, 406, 3352, 3351, 5, 254, 1235..."
693,123541R.msg,"[name, 123541r.msg, elena, kodintseva, <elena....","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 8929, 2954, 3856, 3857, 70, 5, 8930, 8, 48..."


In [21]:
df

,id,tokens,ner_tags,encoded_ner_tags,int_tokens
70,119287R.msg,"[name, 119287r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 9, 11, 12, ..."
76,120421R.msg,"[name, 120421r.msg, d.klebcha@s7.ru, <d.klebch...","[O, O, O, O, O, O, O, O, B-GoodsString, I-Good...","[2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, ...","[0, 72, 73, 74, 75, 76, 5, 77, 78, 79, 80, 8, ..."
88,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 116, 117, 118, 119, 120, 70, 5, 77, 121, 1..."
89,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 116, 117, 118, 119, 120, 70, 5, 77, 121, 1..."
90,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 116, 117, 118, 119, 120, 70, 5, 77, 121, 1..."
...,...,...,...,...,...
2095,125492R.msg,"[name, 125492r.msg, mirgul, sadykova, <mirgul....","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 10885, 1219, 1220, 1221, 70, 5, 10886, 8, ..."
602,123436R.msg,"[name, 123436r.msg, <sales06@yestechnik.com>, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 9633, 3905, 4, 5, 9634, 8, 9, 3907, 312, 1..."
795,123711R.msg,"[name, 123711r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 12355, 3351, 406, 3352, 3351, 5, 254, 1235..."
693,123541R.msg,"[name, 123541r.msg, elena, kodintseva, <elena....","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 8929, 2954, 3856, 3857, 70, 5, 8930, 8, 48..."


In [22]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [23]:
train.head()

,id,tokens,ner_tags,encoded_ner_tags,int_tokens
1554,124789R.msg,"[name, 124789r.msg, <olga.pushkareva@utair.ru>...","[O, O, O, O, O, O, O, O, O, O, B-GoodsString, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, ...","[0, 6832, 4414, 1522, 4415, 4414, 5, 254, 6833..."
259,123118R.msg,"[name, 123118r.msg, =?utf-8?b?0jjqstcw0l3qvtcy...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 3151, 3152, 3153, 3154, 70, 5, 3155, 3156,..."
1120,124145R.msg,"[name, 124145r.msg, =?koi8-r?b?5czj08xf1yd3zmh...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 11362, 467, 468, 70, 5, 11363, 8, 9, 471, ..."
492,123362R.msg,"[name, 123362r.msg, mukhamedkali, tleulenov, <...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 3742, 2921, 2922, 2923, 70, 5, 188, 3743, ..."
2332,125692R.msg,"[name, 125692r.msg, nursultan, momynov, <nursu...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 8554, 6817, 6818, 6819, 70, 5, 77, 8555, 8..."


In [24]:
test.head()

,id,tokens,ner_tags,encoded_ner_tags,int_tokens
1767,125080R.msg,"[name, 125080r.msg, <gennadiy.moskatov@utair.r...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 7545, 4288, 4289, 4290, 4288, 5, 254, 7546..."
874,123820R.msg,"[name, 123820r.msg, =?utf-8?b?0jdqvdc00ydqtdc5...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 4842, 4843, 4844, 70, 5, 193, 77, 4845, 15..."
1175,124236R.msg,"[name, 124236r.msg, <aogdesk@nordstar.ru>, <rf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 11074, 2, 3193, 4856, 5, 6, 11075, 1598, 5..."
1726,125028R.msg,"[name, 125028r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 8586, 3351, 406, 3352, 3351, 5, 254, 8587,..."
215,123043R.msg,"[name, 123043r.msg, mukhamedkali, tleulenov, <...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 2920, 2921, 2922, 2923, 70, 5, 188, 2924, ..."


In [25]:
!pip install keras-crf
!pip install tensorflow==1.15.0 keras==2.2.4

     |████████████████████████████████| 686kB 4.0MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=2304b542557ef0466dee541bc4ee1c8d11b98485800a3eccfe15e3eb44a26122
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
     |████████████████████████████████| 412.3MB 40kB/s 
     |████████████████████████████████| 317kB 29.2MB/s 
     |████████████████████████████████| 3.8MB 30.5MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 512kB 23.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=15144ba6cd3ecdd1f88056e44a9783c7619cb69bd8124c60e48e0196f6ae9cd2
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirem

In [26]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-g6quj320
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-g6quj320
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=020a7b8c8fd2252dc297eac9ac22da99f10cc6b78ca410bdcbcd348ddd2fe474
  Stored in directory: /tmp/pip-ephem-wheel-cache-97tfc9eh/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [27]:
import tensorflow as tf

def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [38]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy
import numpy as np

MAX_WORDS = len(word_to_ix)
EMBEDDING_LENGTH = 512
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
# model.add(Dense(50))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 3004, 512)         6328832   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 3004, 200)         490400    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 3004, 3)           603       
_________________________________________________________________
crf_6 (CRF)                  (None, 3004, 3)           27        
Total params: 6,819,862
Trainable params: 6,819,862
Non-trainable params: 0
_________________________________________________________________


In [39]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=4),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=50,
    batch_size=125,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')

Train on 860 samples, validate on 96 samples
Epoch 1/50
860/860 [==============================] - 131s 153ms/step - loss: 2.8034 - crf_marginal_accuracy: 0.9896 - val_loss: 2.4158 - val_crf_marginal_accuracy: 0.9961
Epoch 2/50
860/860 [==============================] - 126s 146ms/step - loss: 2.0767 - crf_marginal_accuracy: 0.9945 - val_loss: 1.8907 - val_crf_marginal_accuracy: 0.9970
Epoch 3/50
860/860 [==============================] - 125s 146ms/step - loss: 1.6959 - crf_marginal_accuracy: 0.9978 - val_loss: 1.6269 - val_crf_marginal_accuracy: 0.9976
Epoch 4/50
860/860 [==============================] - 127s 147ms/step - loss: 1.2958 - crf_marginal_accuracy: 0.9987 - val_loss: 1.4004 - val_crf_marginal_accuracy: 0.9984
Epoch 5/50
860/860 [==============================] - 125s 146ms/step - loss: 0.9108 - crf_marginal_accuracy: 0.9993 - val_loss: 1.2675 - val_crf_marginal_accuracy: 0.9989
Epoch 6/50
860/860 [==============================] - 131s 152ms/step - loss: 0.6253 - crf_marg

In [42]:
from sklearn_crfsuite import metrics

preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, labels=np.arange(2), digits=3))

              precision    recall  f1-score   support

           0      0.918     0.906     0.912       986
           1      0.963     0.908     0.935      1289

   micro avg      0.943     0.907     0.925      2275
   macro avg      0.940     0.907     0.923      2275
weighted avg      0.943     0.907     0.925      2275

